In [1]:
# Steps to do the VQE
    # 1. Find Molecular Hamiltonian
    # 2. Prepare trial ground state
    # 3. Minimize <H>

In [2]:
import pennylane as qml
from pennylane import numpy as np
from pennylane import qchem

In [4]:
symbols = ["H", "H", "H"]
# Define the geometry of the molecule
coordinates = np.array([[0.0102, 0.0442, 0.0],[0.9867,1.6303,0.0],[1.8720,-0.0085,0.0]])

In [5]:
hamiltonian, qubits = qchem.molecular_hamiltonian(symbols,
                                                  coordinates, 
                                                  charge = 1)

In [6]:
qubits

6

In [7]:
hf = qchem.hf_state(electrons = 2, orbitals = qubits)

In [8]:
hf

tensor([1, 1, 0, 0, 0, 0], requires_grad=True)

In [9]:
num_wires = qubits
dev = qml.device("default.qubit", wires=num_wires)

In [10]:
@qml.qnode(dev)
def exp_energy(state):
    qml.BasisState(np.array(state), wires=range(num_wires))
    return qml.expval(hamiltonian)

In [11]:
exp_energy(hf)

tensor(-1.24655016, requires_grad=True)

In [12]:
def ansatz(params):
    qml.BasisState(hf, wires=range(num_wires))
    qml.DoubleExcitation(params[0], wires=[0,1,2,3])
    qml.DoubleExcitation(params[1], wires=[0,1,4,5])

In [13]:
@qml.qnode(dev)
def cost_function(params):
    ansatz(params)
    return qml.expval(hamiltonian)

In [14]:
cost_function([0.1,0.1])

tensor(-1.26796721, requires_grad=True)

In [15]:
opt = qml.GradientDescentOptimizer(stepsize=0.4)
theta = np.array([0.0,0.0], requires_grad = True)

energy = [cost_function(theta)]
angle = [theta]
max_iterations = 20

for n in range(max_iterations):
    theta, prev_energy = opt.step_and_cost(cost_function, theta)
    
    energy.append(cost_function(theta))
    angle.append(theta)
    
    if n%2 == 0:
        print(f"Step = {n}, Energy = {energy[-1]:.8f} Ha")

Step = 0, Energy = -1.26070025 Ha
Step = 2, Energy = -1.27115671 Ha
Step = 4, Energy = -1.27365804 Ha
Step = 6, Energy = -1.27425241 Ha
Step = 8, Energy = -1.27439362 Ha
Step = 10, Energy = -1.27442718 Ha
Step = 12, Energy = -1.27443517 Ha
Step = 14, Energy = -1.27443707 Ha
Step = 16, Energy = -1.27443752 Ha
Step = 18, Energy = -1.27443763 Ha


In [16]:
print("\n" f"Final ground energy: {energy[-1]:.8f} Ha")
print("\n" f"Final angle parameters: {theta[0]:.8f} {theta[1]:.8f}")


Final ground energy: -1.27443764 Ha

Final angle parameters: 0.19203468 0.19290335


In [17]:
@qml.qnode(dev)
def ground_state(params):
    ansatz(params)
    return qml.state()

In [18]:
ground_state(theta)

tensor([ 0.        +0.j,  0.        +0.j,  0.        +0.j,
        -0.09585862+0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
        -0.09586987+0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.99076743+0.j,  0.        +0.j,  0.        +0.